## Proyecto

In [7]:
#Importamos las librerísas necesarias para llevar a cabo el proyecto
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sb

In [99]:
#Cargamos los datos para este proyecto
datos = np.load('proyecto_data/proyecto_training_data.npy')
datos

array([[2.08500e+05, 7.00000e+00, 8.56000e+02, 8.00000e+00, 2.00300e+03,
        6.50000e+01],
       [1.81500e+05, 6.00000e+00, 1.26200e+03, 6.00000e+00, 1.97600e+03,
        8.00000e+01],
       [2.23500e+05, 7.00000e+00, 9.20000e+02, 6.00000e+00, 2.00100e+03,
        6.80000e+01],
       ...,
       [2.66500e+05, 7.00000e+00, 1.18800e+03, 9.00000e+00, 1.94100e+03,
        6.60000e+01],
       [1.42125e+05, 5.00000e+00, 1.07800e+03, 5.00000e+00, 1.95000e+03,
        6.80000e+01],
       [1.47500e+05, 5.00000e+00, 1.25600e+03, 6.00000e+00, 1.96500e+03,
        7.50000e+01]])

In [98]:
#Se dividen los datos de la siguiente manera:
# 80% de las filas serán datos de entrenamiento y 20% de las filas serán para pruebas

#Datos de entrenamiento
d_entrenamiento = int(0.8 * len(datos))
ds_entrenamiento = datos[:d_entrenamiento]
#datos de validacion
ds_validacion = datos[d_entrenamiento:]

print('Longitud del data set de entrenamiento: ',len(ds_entrenamiento))
print('Longitud del data set de entrenamiento: ',len(ds_validacion))

Longitud del data set de entrenamiento:  1168
Longitud del data set de entrenamiento:  292


In [102]:
#Convertimos el set de datos que vamos a utlizar a dataframe para que sea más fácil identificar cada característica y hacer conversiones
#Agregar nombres a las columnas de acuerdo a la información proporcionada
df_entrenamiento = pd.DataFrame(data = datos, columns = ["PrecioVenta", "CalidadMaterial","PCPerPiso", "THabitaciones", "AConstruccion", "PLFrente"])

#Conversion para los datos de calidad del material, el total de habitaciones y el año de contruscción, ya que son enteros
df_entrenamiento = df_entrenamiento.astype({"CalidadMaterial" : int, "THabitaciones" : int, "AConstruccion" : int})

df_entrenamiento

,PrecioVenta,CalidadMaterial,PCPerPiso,THabitaciones,AConstruccion,PLFrente
0,208500.0,7,856.0,8,2003,65.0
1,181500.0,6,1262.0,6,1976,80.0
2,223500.0,7,920.0,6,2001,68.0
3,140000.0,7,961.0,7,1915,60.0
4,250000.0,8,1145.0,9,2000,84.0
...,...,...,...,...,...,...
1455,175000.0,6,953.0,7,1999,62.0
1456,210000.0,6,2073.0,7,1978,85.0
1457,266500.0,7,1188.0,9,1941,66.0
1458,142125.0,5,1078.0,5,1950,68.0


## Inicia Análisis Exploratorio

In [167]:
media  = np.reshape(np.array(df_entrenamiento.mean(axis=0)),(6,1))
v_max =  np.reshape(np.array(df_entrenamiento.max(axis=0)),(6,1))
v_min = np.reshape(np.array(df_entrenamiento.min(axis=0)),(6,1))
rango = np.reshape(np.array(v_max-v_min),(6,1))
d_estandar = np.reshape(np.array(df_entrenamiento.std(axis=0)),(6,1))

d_exploratorios = np.concatenate((media,v_max,v_min,rango,d_estandar), axis=1)

columnas = df_entrenamiento.columns
df_exploratorios = pd.DataFrame(data = d_exploratorios, columns = ["Media", "VMax","Vmin", "Rango", "DEstandar"], index=columnas)

#Conversion para los datos de calidad del material, el total de habitaciones y el año de contruscción, ya que son enteros
df_exploratorios = df_exploratorios.astype({"VMax" : int, "Vmin" : int, "Rango" : int})

df_exploratorios

,Media,VMax,Vmin,Rango,DEstandar
PrecioVenta,180921.195890,755000,34900,720100,79442.502883
CalidadMaterial,6.099315,10,1,9,1.382997
PCPerPiso,1162.626712,4692,334,4358,386.587738
THabitaciones,6.517808,14,2,12,1.625393
AConstruccion,1971.267808,2010,1872,138,30.202904
PLFrente,70.049958,313,21,292,24.284752
